# Interrogating local Gutenberg docs with an LLM

See: [Talk to your Text files in Vector Databases with GPT-4 and ChromaDB](https://medium.com/@rubentak/unleashing-the-power-of-intelligent-chatbots-with-gpt-4-and-vector-databases-a-step-by-step-8027e2ce9e78)

Import libraries

In [1]:

from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
import os
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
from langchain.document_loaders import DirectoryLoader

Initialize the basic parameters needed for Chroma, considering that the data already exists on disk

In [2]:
persist_directory = 'db2'

# OpenAI embeddings
embedding = OpenAIEmbeddings()

Now we can load the persisted database from disk

In [3]:
vectordb = Chroma(persist_directory=persist_directory,
                   embedding_function=embedding)

In order to get the data out of the database again, we need to create a retriever. This retriever will return all the documents (or chucks) related to the question asked:

In [4]:
# Create retriever
retriever = vectordb.as_retriever()



## Create a Q&A chain
The chain will be created. Consider it to be akin to the prompt in ChatGPT.

In [5]:
# create the chain to answer questions
qa_chain = RetrievalQA.from_chain_type(llm=OpenAI(),
                                  chain_type="stuff",
                                  retriever=retriever,
                                  return_source_documents=True,
                                  verbose=True)

To see how our chatbot will come to the answers that it returns, we can create the following function:

In [6]:
# Cite sources
def process_llm_response(llm_response):
    print(llm_response['result'])
    print('\n\nSources:')
    for source in llm_response["source_documents"]:
        print(source.metadata['source'])

Let's repeat the previous query:

In [7]:
query = "What is the Will?"
llm_response = qa_chain(query)
process_llm_response(llm_response)



> Entering new  chain...

> Finished chain.
 The Will is the fundamental concept in many philosophical theories. It is typically described as an internal, conscious force that drives and motivates us to take certain actions or make certain decisions. The Will is often seen as separate from our emotions and desires, and is considered to be a higher power that guides us in our lives.


Sources:
D:\src\pareidolia\gutenberg\Thus Spake Zarathustra, by Friedrich Nietzsche.txt
D:\src\pareidolia\gutenberg\Beyond Good and Evil, by Friedrich Nietzsche.txt
D:\src\pareidolia\gutenberg\Thus Spake Zarathustra, by Friedrich Nietzsche.txt
D:\src\pareidolia\gutenberg\Beyond Good and Evil, by Friedrich Nietzsche.txt


In [8]:
query = "How did the stoics differ from Plato"
llm_response = qa_chain(query)
process_llm_response(llm_response)



> Entering new  chain...

> Finished chain.
 The Stoics held that material objects alone existed, in contrast to Plato's view that only Ideas or Prototypes of phenomena truly exist. The Stoics also believed in a spiritual force immanent in the material universe, while Plato did not.


Sources:
D:\src\pareidolia\gutenberg\Meditations, by Marcus Aurelius.txt
D:\src\pareidolia\gutenberg\Meditations, by Marcus Aurelius.txt
D:\src\pareidolia\gutenberg\Meditations, by Marcus Aurelius.txt
D:\src\pareidolia\gutenberg\Meditations, by Marcus Aurelius.txt


In [10]:
query = "How can man be freed of the constraints of Ethics?"
llm_response = qa_chain(query)
process_llm_response(llm_response)



> Entering new  chain...

> Finished chain.
 A philosopher would argue that man's greatness and worth can be determined by the amount and variety of things he can bear and take responsibility for, and the extent to which he can stretch his responsibility.


Sources:
D:\src\pareidolia\gutenberg\Beyond Good and Evil, by Friedrich Nietzsche.txt
D:\src\pareidolia\gutenberg\Beyond Good and Evil, by Friedrich Nietzsche.txt
D:\src\pareidolia\gutenberg\Beyond Good and Evil, by Friedrich Nietzsche.txt
D:\src\pareidolia\gutenberg\Beyond Good and Evil, by Friedrich Nietzsche.txt


In [9]:
query = "What did Nietzche think of the stoics?"
llm_response = qa_chain(query)
process_llm_response(llm_response)



> Entering new  chain...

> Finished chain.
 Nietzsche thought that the Stoics forced themselves to see nature falsely, and that their philosophy was a form of self-tyranny. He also noted that their system of physics was materialism with an infusion of pantheism, and that they believed that virtue was the highest good and the source of happiness.


Sources:
D:\src\pareidolia\gutenberg\Beyond Good and Evil, by Friedrich Nietzsche.txt
D:\src\pareidolia\gutenberg\Meditations, by Marcus Aurelius.txt
D:\src\pareidolia\gutenberg\Meditations, by Marcus Aurelius.txt
D:\src\pareidolia\gutenberg\Meditations, by Marcus Aurelius.txt
